In [14]:
import numpy as np
import pandas as pd 
train_df = pd.read_csv('../input/projectdata/train.csv')
test_df = pd.read_csv('../input/projectdata/test.csv')
#train_df.fillna(method='ffill',inplace=True)
#train_df = train_df[~pd.isna(train_df).any(axis=1)].reset_index(drop=True)#xoá hàng có chứa NaN
train_df.info()

In [15]:
keypoints_df = train_df.drop(train_df.columns[0:9],axis = 1)
keypoints_df = keypoints_df.drop(keypoints_df.columns[2::3],axis = 1)
keypoints_df = keypoints_df.drop(keypoints_df.columns[80:],axis = 1)
y_train_size=train_df.columns[2:4]
y_train = np.array(keypoints_df,dtype='float')
y_train_size=train_df[['height','width']]
y_train_size=np.array(y_train_size,dtype='float')
for i in range(0,y_train.shape[1],2):
    for j in range(y_train.shape[0]):
        y_train[j][i]=y_train[j][i]/y_train_size[j][1]*240
        y_train[j][i+1]=y_train[j][i+1]/y_train_size[j][0]*240
print(y_train.shape)

In [16]:
keytest_df = test_df.drop(test_df.columns[0:9],axis = 1)
keytest_df = keytest_df.drop(keytest_df.columns[2::3],axis = 1)
keytest_df = keytest_df.drop(keytest_df.columns[80:],axis = 1)
y_test = np.array(keytest_df,dtype='float')
y_test_size=test_df[['height','width']]
y_test_size=np.array(y_test_size,dtype='float')
for i in range(0,y_test.shape[1],2):
    for j in range(y_test.shape[0]):
        y_test[j][i]=y_test[j][i]/y_test_size[j][1]*240
        y_test[j][i+1]=y_test[j][i+1]/y_test_size[j][0]*240
print(y_test.shape)

In [17]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
datagen = ImageDataGenerator(rescale=1./255)
data1=datagen.flow_from_directory('../input/dataset/datasets',
                                          target_size=(240,240),batch_size=5,
                                          interpolation="lanczos",shuffle=False)
imageArr = []
for i in range(182):
        img=load_img('../input/dataset/datasets/'+data1.filenames[i],
                 target_size=(240,240))
        img=img_to_array(img)
        img=img.reshape(1,240,240,3)
        img=img.astype('float')
        img/=255
        imageArr.append(img)
x_train = np.array(imageArr,dtype='float')
x_train = x_train.reshape(-1,240,240,3)
imageArr_test = []
for i in range(182,208):
    img1=load_img('../input/dataset/datasets/'+data1.filenames[i],
             target_size=(240,240))
    img1=img_to_array(img1)
    img1=img1.reshape(1,240,240,3)
    img1=img1.astype('float')
    img1/=255
    imageArr_test.append(img1)
x_test = np.array(imageArr_test,dtype='float')
x_test = x_test.reshape(-1,240,240,3)

In [18]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
a=20
img=load_img('../input/dataset/datasets/'+data1.filenames[a],
             target_size=(240,240))
plt.imshow(img)
for i in range(0,80,2):
    plt.plot(y_train[a][i],
             y_train[a][i+1],
             'b*',markersize=5)
plt.xticks([])
plt.yticks([])
plt.show()

In [19]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
a=20
img=load_img('../input/dataset/datasets/'+data1.filenames[a+182],
             target_size=(240,240))
plt.imshow(img)
for i in range(0,80,2):
    plt.plot(y_test[a][i],
             y_test[a][i+1],
             'b*',markersize=5)
plt.xticks([])
plt.yticks([])
plt.show()

In [20]:
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D,MaxPooling2D,BatchNormalization, Flatten, Dense, Dropout,GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import RMSprop
model = Sequential()
model.add(Convolution2D(32,(3,3),padding='same',use_bias=False, input_shape=(240,240,3)))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(32,(3,3),padding='same',use_bias = False))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64,(3,3),padding='same',use_bias = False))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# model.add(BatchNormalization())
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(512, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(512, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(1024, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())

model.add(Convolution2D(1024, (3,3),padding='same',use_bias=False))
model.add(LeakyReLU(alpha = 0.1))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
#model.add(GlobalAveragePooling2D())
model.add(Dense(1024,activation='relu'))
# model.add(Dropout(0.1))
model.add(Dense(80))
model.compile(optimizer='Adamax',loss='mean_squared_error',metrics=['accuracy'])
model.summary()

In [21]:
history=model.fit(x_train,y_train,batch_size=4,epochs=500,validation_data=(x_test,y_test))

In [22]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('accuracy')
plt.ylabel('epoch')
plt.legend(['Train','Validation'],loc='upper left')
plt.show()

In [23]:
model.save('project.h5')

In [24]:
from tensorflow.keras.models import load_model
model_1=load_model('./project.h5')

In [25]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
img=load_img('../input/predict3/1.jpg',target_size=(240,240))
plt.imshow(img)
img=img_to_array(img)
img=img.reshape(1,240,240,3)
img=img.astype('float')
img/=255
y=model_1.predict(img)

for i in range(0,80,2):
    plt.plot(y[0][i],
             y[0][i+1],
             'b*',markersize=5)
plt.xticks([])
plt.yticks([])
plt.show()

In [26]:
y